In [1]:
import joblib
import numpy as np
import re
import math
import statistics as stats
from collections import Counter
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

/Users/huynhcongbang/ai_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

PUNCT_RE = re.compile(r"[^\w\s]", re.UNICODE)
STOPWORDS = set(["the", "be", "to", "of", "and", "a", "in", "that", "have", "i", 
                 "it", "for", "not", "on", "with", "he", "as", "you", "do", "at"])

def count_syllables(word):
    word = word.lower()
    if len(word) <= 3: return 1
    word = re.sub(r'(?:[^laeiouy]es|ed|[^laeiouy]e)$', '', word)
    word = re.sub(r'^y', '', word)
    return max(1, len(re.findall(r'[aeiouy]{1,2}', word)))

def shannon_entropy(text):
    if not text: return 0
    counts = Counter(text)
    total = len(text)
    return -sum((cnt / total) * math.log2(cnt / total) for cnt in counts.values())

def extract_features_pro(text):
    words = text.split()
    sentences = [s for s in re.split(r"[.!?]", text) if s.strip()]
    if not sentences: sentences = [text]
    avg_sent_len = len(words) / max(1, len(sentences))
    sent_lens = [len(s.split()) for s in sentences]
    burstiness = stats.pstdev(sent_lens) / avg_sent_len if avg_sent_len > 0 else 0
    num_syllables = sum(count_syllables(w) for w in words)
    flesch_score = 206.835 - 1.015 * avg_sent_len - 84.6 * (num_syllables / max(1, len(words)))
    stop_count = sum(1 for w in words if w.lower() in STOPWORDS)
    
    return {
        "len_words": len(words),
        "ttr": len(set(words)) / max(1, len(words)),
        "punct_ratio": len(PUNCT_RE.findall(text)) / max(1, len(text)),
        "avg_sent_len": avg_sent_len,
        "burstiness": burstiness,
        "entropy": shannon_entropy(text),
        "flesch_score": flesch_score,
        "stopword_ratio": stop_count / max(1, len(words))
    }

In [3]:
class PPLScorer:
    def __init__(self, model_name="distilgpt2"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.model.eval()
    
    def perplexity(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            loss = self.model(**inputs, labels=inputs["input_ids"]).loss
        return float(torch.exp(loss))

def load_model(path="aidetect_model_pro.joblib"):
    data = joblib.load(path)
    print("✅ Model Pro loaded successfully.")
    return data["pipe"], data["feat_names"], PPLScorer()

def predict_text(text, pipe, feat_names, scorer):
    f = extract_features_pro(text)
    f["ppl"] = scorer.perplexity(text)
    # Đảm bảo thứ tự feature đúng như lúc train
    vector = np.array([f[name] for name in feat_names]).reshape(1, -1)
    return pipe.predict_proba(vector)[0, 1]

In [ ]:
pipe, feat_names, scorer = load_model()

print("\n=== AI TEXT DETECTOR  ===")
print("Nhập đoạn văn tiếng Anh để kiểm tra. Gõ 'exit' để thoát.\n")

while True:
    text = input(">> Nhập văn bản: ").strip()
    if not text: continue
    if text.lower() in ["exit", "quit"]: break
    
    try:
        prob_ai = predict_text(text, pipe, feat_names, scorer)
        
        # Vẽ thanh tiến độ cho đẹp
        bar_len = 20
        filled = int(prob_ai * bar_len)
        bar = "█" * filled + "░" * (bar_len - filled)
        
        print(f"\nKẾT QUẢ: [{bar}] {prob_ai*100:.1f}% AI")
        if prob_ai > 0.5:
            print("=> Khả năng cao là AI viết 🤖\n")
        else:
            print("=> Khả năng cao là NGƯỜI viết 🧑\n")
            
    except Exception as e:
        print(f"Lỗi: {e}")

✅ Model Pro loaded successfully.

=== AI TEXT DETECTOR  ===
Nhập đoạn văn tiếng Anh để kiểm tra. Gõ 'exit' để thoát.



>> Nhập văn bản:  In this study, we propose a planetary gear degradation state recognition method based on features with multiple perspectives and LLTSA. Firstly, feature extraction is used to collect vibration data from the gears. Then, time-domain analysis is conducted to characterize the degradation of the gears. Data mining is performed to discover the relevant features of the gear vibrations, and dimensionality reduction is used to select the most significant features. Finally, LLTSA is used to build a recognition model based on the extracted features. The proposed method effectively captures the degradation characteristics of planetary gears and provides an accurate recognition of the gear's state. This research provides a valuable contribution to the field of gear degradation recognition, and further research can be conducted to improve the accuracy and efficiency of the proposed method.


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.



KẾT QUẢ: [█████████░░░░░░░░░░░] 49.3% AI
=> Khả năng cao là NGƯỜI viết 🧑



>> Nhập văn bản:  This paper proposes an active learning method for gearbox fault diagnosis, which is based on the concepts of uncertainty and complexity. The approach is centered on the use of supervised learning algorithms to train models with radio frequency data that can help to understand and identify different fault states. One of the key features of the method is the way it selectively chooses samples to train the models, based on their level of uncertainty and complexity. This is achieved through the application of a complexity theory-inspired approach that seeks to balance the trade-off between model accuracy and computational efficiency. The experimental results demonstrate that the proposed method outperforms existing approaches in terms of both diagnostic accuracy and computational efficiency. In summary, this paper provides a novel approach to fault diagnosis that combines the strengths of supervised learning, uncertainty, and complexity theory to improve the performance o

...


KẾT QUẢ: [███████████████████░] 97.7% AI
=> Khả năng cao là AI viết 🤖



>> Nhập văn bản:  It’s hard to imagine what our lives would be like without science and technology. They aren't just subjects in a textbook; they shape everything we do, from the smartphones in our pockets to the medicine that keeps us healthy. Science gives us the "why," and technology gives us the "how" to solve real-world problems. With exciting changes like AI and clean energy happening right now, we aren't just dreaming of a better future—we are actively building it.



KẾT QUẢ: [██████░░░░░░░░░░░░░░] 34.0% AI
=> Khả năng cao là NGƯỜI viết 🧑



>> Nhập văn bản:  The dataset is divided according to 60% and 40% and used as training set and test set respectively, thenumber of training rounds is set to 10, the graphic card used for the experiment is 3090, the GPUis32G, and the experiment is conducted using python 3.10. The changes in accuracy and loss duringthetraining process were recorded and the results are shown in Fig. 5.



KẾT QUẢ: [░░░░░░░░░░░░░░░░░░░░] 0.9% AI
=> Khả năng cao là NGƯỜI viết 🧑

